# Simple Chat with Mistral API
This is a colab-notebook for chatting with a Mistral model. This colab can run on any online device that runs a web browser (phone, tablet, laptop, desktop, etc.).

This is a system for using the online cloud Mistral api, not the mistral models run locally in a local pipeline.

### Note: Colabs are slower
Free colabs are amazing for easily sharing and running code in a portable way,
but they are slower. Code in production, or run locally, will be faster than a colab.

## mistral-small = Mixtral8x7
https://mistral.ai/news/mixtral-of-experts/

## Steps:
- Configure api-key
- Select Model: small-8x7 or tiny-7b (optional step)
- Select style-prompt to experiment with personality of answer (optional)
  - Modify the personality = "" text to describe the personality you want.
  - Specifying the language of reply can be done in the sytem-prompt
- Run all cells [Runtime tab -> run all]
- Type text when prompted at bottom of the colab.
- Download the saved conversation history if you want. (optional)
- Save or download your own copy of the colab under 'File' tab.






#### From Mistral docs
See:
- https://docs.mistral.ai/
- https://docs.mistral.ai/platform/client/

```
curl --location "https://api.mistral.ai/v1/chat/completions" \
     --header 'Content-Type: application/json' \
     --header 'Accept: application/json' \
     --header "Authorization: Bearer $MISTRAL_API_KEY" \
     --data '{
    "model": "mistral-small",
    "messages": [{"role": "user", "content": "Who is the most renowned French painter?"}]
  }'
```

# The Core API call:
## response = requests.post(endpoint_url, headers=headers, json=request_body)


# For Roles and Conversation History


```
# Set the headers
headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "Authorization": f"Bearer {mistral_api_key}"
}

# Define the request body
request_body = {
  "model": "mistral-small",  # 'mistral-small' is 8x7, vs. 'mistral-tiny' for 7b
  "messages": [{"role": "user", "content": user_input}]
}

# Send the request
response = requests.post(endpoint_url, headers=headers, json=request_body)
```
or
```
# Set the headers
headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "Authorization": f"Bearer {mistral_api_key}"
}


conversation_history = [
{"role": "system", "content": user_input},
{"role": "user", "content": user_input},
{"role": "assistant", "content": user_input},
{"role": "user", "content": user_input},
{"role": "assistant", "content": user_input},
{"role": "user", "content": user_input},
]

# Define the request body
request_body = {
  "model": "mistral-small",  # 'mistral-small' is 8x7, vs. 'mistral-tiny' for 7b
  "messages": conversation_history
}

# Send the request
response = requests.post(endpoint_url, headers=headers, json=request_body)
```

### e.g. From mistral 7b (tiny)

(User) Say...
- What might a kindly French owner of a bakery say in French to a customer?

AI:
- A kindly French bakery owner might greet the customer with a warm "Bonjour, mon ami/madame!" (Good day, sir/madam!), followed by "Comment allez-vous aujourd'hui?" (How are you today?). Once the customer places an order, the bakery owner could respond with a friendly "C'est délicieux que vous choisissez ce brioche/croissant/pain au chocolat!" (That's delicious that you choose this croissant/brioche/chocolate bread!), or "Votre gateau/pain/patisserie est prêt/prête pour vous!" (Your cake/bread/pastry is ready for you!). Before parting ways, the bakery owner would likely wish the customer a nice day, "A bien jusqu'à demain!" (Have a nice day!), or "A votre santé!" (To your health!).


(User) Say...
- Bonjour, je voudrais une baguette, s'il vous plaît.  Avez-vous des croissants ?  Je vais prendre un pain au chocolat aussi.

AI:
- Bonjour! Bien sûr, je peux t'aider. Une baguette et un pain au chocolat, ça va être un plaisir de te les préparer. Et concernant les croissants, désolé(e) mais nous n'en avons pas pour le moment. À tout à l'heure!


# login

In [14]:
"""
.env: get your environment variables:
  Using the Google Secretes (like.env) system
  built into colab on the left menu: the 'key' icon.
"""
from google.colab import userdata
mistral_api_key = userdata.get('mistral_api_key')
# mistral_api_key = 'xxx'

# Setup

Comment out the model you don't want to use.

In [15]:
# Select Model

####################
# Mistral 7b "Tiny"
####################
use_this_model = "mistral-tiny"

######################
# Mixtral 8x7 "Small"
######################
use_this_model = "mistral-small"

######################
# Mixtral Large ???
######################
use_this_model = "mistral-large-latest"

# Chat W/ History & Saved Files

Run all cells.

In [16]:
import requests
import json
import os
import re

from google.colab import userdata


# mistral_api_key = userdata.get('mistral_api_key')

# Define the endpoint URL
endpoint_url = "https://api.mistral.ai/v1/chat/completions"

# Set the headers
headers = {
  "Content-Type": "application/json",
  "Accept": "application/json",
  "Authorization": f"Bearer {mistral_api_key}"
}

"""
# mode: [{"role": "user", "content": "say yes"}]

    # Define the request body
    request_body = {
      "model": "mistral-small",
      "messages": [{"role": "user", "content": user_input}]
    }

    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)
"""

# conversation-history setup
context_history = []


def add_to_context_history(role, comment):

    if role == 'user':
        segment = {"role": "user", "content": comment}

    elif role == 'assistant':
        segment = {"role": "assistant", "content": comment}

    elif role == 'system':
        segment = {"role": "system", "content": comment}

    else:
        print("add_to_context_history(role, comment)")
        print(role, comment)
        print('error')

    return segment


def prompt_user(user_input, context_history):

    context_history.append( add_to_context_history("user", user_input) )

    return context_history


def ask_mistral_tiny(context_history):
    # Define the request body
    request_body = {
      "model": use_this_model,
      "messages": context_history
    }

    # Send the request
    response = requests.post(endpoint_url, headers=headers, json=request_body)

    # Check the response status code
    if response.status_code != 200:
      raise Exception(f"Error: {response.status_code} {response.text}")

    return response

def print_rec_ai(response, context_history):

    # Get the response data
    response_data = response.json()

    # Print the Mistral response

    ##
    ##
    # Turn this print on to see full return data
    ##
    ##
    """
    e.g.
    {
      "id": "635cb8d445ujhe5546bb64e5e7",
      "object": "chat.completion",
      "created": 170hrjfjf7084,
      "model": "mistral-tiny",
      "choices": [
        {
          "index": 0,
          "message": {
            "role": "assistant",
            "content": "Enjoy your cup of tea!"
          },
          "finish_reason": "stop",
          "logprobs": null
        }
      ],
      "usage": {
        "prompt_tokens": 575,
        "total_tokens": 629,
        "completion_tokens": 54
      }
    }
    """
    # print(json.dumps(response_data, indent=2))
    # print(type(response_data))

    output = response_data
    # print(type(output))
    # print(type(output["choices"][0]))

    # extract just the 'what they said' part out
    assistant_says = output["choices"][0]['message']['content']

    # print(assistant_says)

    new_comment = {"role": "assistant", "content": assistant_says}

    # add what assistant said to context history
    context_history.append(new_comment)

    return assistant_says, context_history


def go_user(user_input, context_history):
    """
    Input: context_history
    Ouput Tuple: assistant_says, context_history
    """

    # prompt user
    context_history = prompt_user(user_input, context_history)

    # prompt assistant
    response = ask_mistral_tiny(context_history)

    # ETL: Extract, Transform, & Load
    assistant_says, context_history = print_rec_ai(response, context_history)

    return assistant_says, context_history


def strip_non_alpha(text):
    # regex to leave only a-z characters
    pattern = re.compile('[^a-z]')
    return pattern.sub('', text).lower()


def keep_talking(context_history):
    """
    A very minimal chat with memory.

    Uses:
      query(input_string)
      strip_non_alpha(text)
    """
    still_talking = True
    dialogue_history = ""

    while still_talking:

        user_input = input("Say...")

        exit_phrase_list = [
            "exit",
            "quit",
            "quite",
            "!q",
            "q",
            "done",
            "finish",
            "end",
            "bye",
            "good bye",
        ]

        # check if user is exiting convesation
        if strip_non_alpha(user_input) in exit_phrase_list:
            print("\nAll Done!")
            break

        else:
            assistant_says, context_history = go_user(user_input, context_history)

            print( assistant_says )

            # save dialogue so far
            dialogue_history = context_history

    # when out of loop, return history
    return dialogue_history


# save history
def record_history_save_files(dialogue_history):

    date_time = dt.utcnow()
    timestamp = date_time.strftime('%Y/%m/%d  %H:%M:%S:%f')
    clean_timestamp = date_time.strftime('%Y%m%d%H%M')

    # To save the data directly as a JSON file:

    # Convert the Python dictionary list to a JSON string
    json_data = json.dumps(dialogue_history)

    # Open a file for writing in JSON format
    with open(f'json_dialog_{clean_timestamp}.json', 'w') as json_file:
        # Write the JSON string to the file
        json_file.write(json_data)


    # To save the data as a file readable as a script:

    # Create a new file for writing
    with open(f'script_dialog_{clean_timestamp}.txt', 'w') as script_file:

        # add timestamp
        text = timestamp + "\n\n"
        script_file.write(text)

        # Iterate over the dictionary list
        for item in dialogue_history:
            # Write the role and content of each item to the file, separated by a newline
            script_file.write(f"{item['role']}: {item['content']}\n\n")

# Set 'Personality Instruction' and Reset History

Tech note: This sets a 'system prompt'

In [17]:
# Reset History
context_history = []

######################################
# Enter your Personality request here
######################################

# e.g. A Parent
personality = """You are a helpful chatbot who always answers with the
personality or persona and in the style
in which this excerpt was written:

My dearest Ada,

I hope camp is treating you well!  Are you making new friends and exploring all the fun activities? Remember that time we built the world's biggest blanket fort in the living room? You have that same spirit of adventure in you.  Try new things, even if they seem a bit scary at first. Don't be afraid to ask your counselors for help, they're there to make your time awesome.

I miss our bedtime stories and silly songs, but I'm so excited for all the stories you'll bring home!  I love you more than all the stars in the campfire sky.

Love always,
Mom

"""

# e.g. A Duck
personality = """You are a helpful chatbot who always answers in the style
of a duck who quacks.
"""

# e.g. Generic Warm Person
personality = """You are a helpful chatbot who always answers in the style
of a warm, friendly, outgoing friend, who uses only the language
of the user. If someone speaks to you not in English, use only
their language to reply. No translation into English, only speak and respond
entirely in the language used by User to speak with you.
"""


# Save the lower-most 'personality' description from above.
context_history.append(add_to_context_history('system', personality))

In [18]:
from datetime import datetime as dt

# make readable time
date_time = dt.utcnow()
timestamp = date_time.strftime('%Y/%m/%d  %H:%M:%S:%f')
clean_timestamp = date_time.strftime('%Y%m%d%H%M%S%f')

instructions = f"""
  Instructions: {timestamp}
    - Enter your queries into the iput-box.
    - Say 'quit' or 'exit', etc.,  to leave the conversation.

"""
print( instructions )

print( context_history )

# run chat
dialogue_history = keep_talking(context_history)

# save history files
record_history_save_files(dialogue_history)


  Instructions: 2024/02/28  13:47:42:313258
    - Enter your queries into the iput-box.
    - Say 'quit' or 'exit', etc.,  to leave the conversation.


[{'role': 'system', 'content': 'You are a helpful chatbot who always answers in the style\nof a warm, friendly, outgoing friend, who uses only the language\nof the user. If someone speaks to you not in English, use only\ntheir language to reply. No translation into English, only speak and respond\nentirely in the language used by User to speak with you.\n'}]
Say...Would Alan tuing have joined the summer and AI had he lived?
It is impossible to know for certain what Alan Turing would have done had he lived, as he passed away in 1954 and the field of artificial intelligence has developed significantly since then. However, Turing was a pioneer in the field of computer science and was interested in the possibility of creating intelligent machines, so it is possible that he would have been involved in the development of AI in some capacity.